In [1]:
import os
os.chdir("..")
print(os.getcwd())

/Users/rosinatorres/Documents/PhD/Project/Code/MS2LDA/MS2LDA


In [2]:
from Preprocessing.load_and_clean import load_mgf
from Preprocessing.load_and_clean import clean_spectra

from Preprocessing.generate_corpus import features_to_words
from Preprocessing.generate_corpus import combine_features

In [3]:
spectra_path = "test_data/pos_ache_inhibitors_pesticides.mgf"
spectra = load_mgf(spectra_path)
cleaned_spectra = clean_spectra(spectra)
fragment_words, loss_words = features_to_words(cleaned_spectra)
feature_words = combine_features(fragment_words, loss_words)

In [4]:
from MS2LDA.modeling import define_model
from MS2LDA.modeling import train_model
from MS2LDA.modeling import extract_motifs
from MS2LDA.modeling import create_motif_spectra

In [5]:
model_parameters = {}
ms2lda = define_model(n_motifs=88, model_parameters=model_parameters)
train_parameters = {"parallel": 4}
trained_ms2lda = train_model(ms2lda, feature_words, iterations=300, train_parameters=train_parameters)

In [6]:
from tomotopy.coherence import Coherence
import tomotopy as tp
coherence=tp.coherence.Coherence(trained_ms2lda, coherence='c_v')
print(coherence.get_score())

0.561083340788768


In [7]:
# retrieves motif from LDA model and converts it into a matchms spectrum object
motifs = extract_motifs(trained_ms2lda, top_n=20)
motif_spectra = create_motif_spectra(motifs)

In [8]:
import tmplot as tmp
phi = tmp.get_phi(trained_ms2lda)
phi.head()

topics,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
words,,,,,,,,,,,,,,,,,,,,,
frag@124.98,0.019025,0.020565,0.016845,0.014053,0.015132,0.019396,0.019001,0.016410,0.017143,0.013784,...,0.012867,0.019318,0.014865,0.018630,0.018852,0.017405,0.017818,0.018309,0.019150,0.015456
frag@96.95,0.013623,0.016176,0.015692,0.014053,0.017496,0.018214,0.012514,0.014099,0.013669,0.013317,...,0.015623,0.016758,0.018116,0.015904,0.017013,0.019008,0.014810,0.014981,0.015081,0.015002
frag@77.04,0.013858,0.009475,0.014538,0.011946,0.012768,0.014430,0.012050,0.015717,0.014364,0.010981,...,0.013785,0.012569,0.009988,0.013859,0.011266,0.012596,0.010877,0.014030,0.011251,0.010229
frag@142.99,0.012684,0.013634,0.014769,0.012414,0.012532,0.012774,0.012977,0.011326,0.009499,0.011915,...,0.011718,0.011405,0.009988,0.012723,0.011036,0.011680,0.010877,0.011652,0.009097,0.009775
frag@114.96,0.009631,0.012017,0.011769,0.010541,0.011113,0.011355,0.011355,0.012944,0.008804,0.010280,...,0.011259,0.012336,0.010453,0.012723,0.011266,0.012825,0.010182,0.009988,0.012687,0.006820


In [9]:
topics_coords = tmp.prepare_coords(trained_ms2lda)
topics_coords.head()

,x,y,topic,size,label
0,0.387890,0.361225,0,1.115309,0
1,0.262047,0.155091,1,1.150097,1
2,-0.150915,0.103056,2,1.155057,2
3,0.060351,0.880812,3,1.129557,3
4,-1.104322,1.065612,4,1.114089,4


In [10]:
tmp.plot_scatter_topics(topics_coords, size_col='size', label_col='label')

alt.LayerChart(...)

In [11]:
terms_probs = tmp.calc_terms_probs_ratio(phi, topic=10, lambda_=1)
tmp.plot_terms(terms_probs)

alt.Chart(...)

In [12]:
tmp.report(trained_ms2lda, docs=feature_words, height=400, width=250)

In [ ]:
#Quick way of savin the LDA output into a list (pickle)
import pickle

folder_path = "test_data"
# Save the list to a file
with open(os.path.join(folder_path, 'motifs.pkl'), 'wb') as f:
    pickle.dump(motifs, f)

In [ ]:
folder_path = "test_data"
# Save the list to a file
with open(os.path.join(folder_path, 'motifs_spectra.pkl'), 'wb') as f:
    pickle.dump(motif_spectra, f)

In [ ]:
folder_path = "test_data"
# Save the list to a file
with open(os.path.join(folder_path, 'corpus.pkl'), 'wb') as f:
    pickle.dump(feature_words, f)

In [ ]:
import joblib
import os

folder_path = "test_data"
#Load the motifs from MS2LDA Run_Template as a list
lda_model = joblib.load(os.path.join(folder_path, 'motifs.pkl'))

motifs_spectra = joblib.load(os.path.join(folder_path, 'motifs_spectra.pkl'))

corpus = joblib.load(os.path.join(folder_path, 'corpus.pkl'))

In [ ]:
import pickle as pkl

with open(os.path.join(folder_path, 'motifs.pkl'), 'rb') as file:
    model = pkl.load(file)

print(model)

In [ ]:
import tmplot as tmp
phi = tmp.get_phi(model)
phi.head()

In [ ]:
topics_coords = tmp.prepare_coords(model)
topics_coords.head()

In [ ]:
motifs_spectra[1].plot()